# Separando as amostras em Treino, Validação e Teste

Este notebook tem como objetivo dividir as amostras em treino, validação e teste.

Ao invés de usar uma ferramenta pronta (como a função _train_test_split_ do *sklearn*), mostro aqui o passo a passo para balancear o dataset e fazer a divisão em treino, validação e teste.

Uma boa prática que também adotei aqui é verificar se existem amostras repetidas.

In [1]:
import numpy as np
from os import listdir, mkdir
from os.path import isfile, join, isdir
import pickle

In [2]:
X_files = sorted([join('../data/', f) for f in listdir('../data/') if isfile(join('../data/', f)) and 'trainX' in f])
Y_files = sorted([join('../data/', f) for f in listdir('../data/') if isfile(join('../data/', f)) and 'trainY' in f])

In [3]:
# Agrupando todas as amostras em uma única lista
all_samples, all_labels = None, None
# Percorre par de arquivos
for fx,fy in zip(X_files,Y_files):
    sample_x = np.load(fx)
    label = np.load(fy)
    if all_samples is None:
        all_samples = sample_x
        all_labels = label
    else:
        all_samples = np.concatenate((all_samples, sample_x))
        all_labels = np.concatenate((all_labels, label))

In [4]:
r# Embaralhando amostras para evitar viés dos dados fornecidos
random_seed = 123
np.random.seed(random_seed)
indexes = np.arange(all_samples.shape[0])
np.random.shuffle(indexes)
all_samples = all_samples[indexes]
all_labels = all_labels[indexes]

In [5]:
# Converte em listas
all_samples = list(all_samples)
all_labels = list(all_labels)

In [6]:
# Verifica possíveis amostras repetidas com checksum
checksum = {}
for i, sample in enumerate(all_samples):
    _sum = sample.sum()
    checksum.setdefault(_sum, [])
    checksum[_sum].append(i)
# Para cada possível amostra repetida, verifica se está realmente duplicada
idx_to_remove = []
for k, v in checksum.items():
    if len(v) == 1:
        continue
    # Duplicidade confirmada
    if (all_samples[v[0]] == all_samples[v[1]]).all():
        idx_to_remove.append(v[0])
# Inverte indexes para não remover amostras erradas com o pop
idx_to_remove.sort(reverse=True)
[all_samples.pop(idx) for idx in idx_to_remove]
[all_labels.pop(idx) for idx in idx_to_remove]
print(f'Amostras duplicadas removidas: {idx_to_remove}')

Amostras duplicadas removidas: [4380, 3410, 1710, 1113]


In [7]:
# Divide em amostras positivas e negativas
all_pos_samples = [sample for i, sample in enumerate(all_samples) if all_labels[i][0] == 0.]
all_neg_samples = [sample for i, sample in enumerate(all_samples) if all_labels[i][0] == 1.]
len(all_pos_samples), len(all_neg_samples)

(2224, 4444)

In [8]:
# Balanceamento das classes (50% positivas e 50% negativas)
min_samples = min(len(all_pos_samples), len(all_neg_samples))
total_samples = 2*min_samples

In [9]:
# Obtém quantidade de amostras para treino, validação e teste
# 60% treino 
qty_samples_train = round(0.6*total_samples)
qty_samples_train_pos = round(qty_samples_train/2)
qty_samples_train_neg = qty_samples_train - qty_samples_train_pos
# 20% validação
qty_samples_val = round(0.2*total_samples)
qty_samples_val_pos = round(qty_samples_val/2)
qty_samples_val_neg = qty_samples_val - qty_samples_val_pos
# 20% teste
qty_samples_test = round(0.2*total_samples)
qty_samples_test_pos = round(qty_samples_test/2)
qty_samples_test_neg = qty_samples_test - qty_samples_test_pos
# Quantidade total de amostras
total_samples = qty_samples_train + qty_samples_val + qty_samples_test
# Treino
print(f'Treino: {qty_samples_train} ({100*qty_samples_train/total_samples:.4f}%)')
print(f'\t* positivas: {qty_samples_train_pos} ({100*qty_samples_train_pos/qty_samples_train:.4f}%)')
print(f'\t* negativas: {qty_samples_train_neg} ({100*qty_samples_train_neg/qty_samples_train:.4f}%)')
# Validação
print(f'Validação: {qty_samples_val} ({100*qty_samples_val/total_samples:.4f}%)')
print(f'\t* positivas: {qty_samples_val_pos} ({100*qty_samples_val_pos/qty_samples_val:.4f}%)')
print(f'\t* negativas: {qty_samples_val_neg} ({100*qty_samples_val_neg/qty_samples_val:.4f}%)')
# Teste
print(f'Teste: {qty_samples_test} ({100*qty_samples_test/total_samples:.4f}%)')
print(f'\t* positivas: {qty_samples_test_pos} ({100*qty_samples_test_pos/qty_samples_test:.4f}%)')
print(f'\t* negativas: {qty_samples_test_neg} ({100*qty_samples_test_neg/qty_samples_test:.4f}%)')


Treino: 2669 (59.9910%)
	* positivas: 1334 (49.9813%)
	* negativas: 1335 (50.0187%)
Validação: 890 (20.0045%)
	* positivas: 445 (50.0000%)
	* negativas: 445 (50.0000%)
Teste: 890 (20.0045%)
	* positivas: 445 (50.0000%)
	* negativas: 445 (50.0000%)


In [10]:
# Dividindo datasets de treino, validação e teste
# Treino
samples_train = all_pos_samples[0:qty_samples_train_pos] + \
                all_neg_samples[0:qty_samples_train_neg]
labels_train = ([1]*qty_samples_train_pos) + \
               ([0]*qty_samples_train_neg)
# Validação
samples_validation = all_pos_samples[qty_samples_train_pos:qty_samples_train_pos+qty_samples_val_pos] + \
                     all_neg_samples[qty_samples_train_neg:qty_samples_train_neg+qty_samples_val_neg]
labels_validation = ([1]*qty_samples_val_pos) + ([0]*qty_samples_val_neg)
# Teste
samples_test = all_pos_samples[qty_samples_train_pos+qty_samples_val_pos:qty_samples_train_pos+qty_samples_val_pos+qty_samples_test_pos] + \
               all_neg_samples[qty_samples_train_neg+qty_samples_val_neg:qty_samples_train_neg+qty_samples_val_neg+qty_samples_test_neg]
labels_test = ([1]*qty_samples_test_pos) + ([0]*qty_samples_test_neg)

In [11]:
# Embaralhando amostras
import random
random.seed(123)
# treino
idxes = list(range(len(samples_train)))
random.shuffle(idxes)
samples_train = [samples_train[i] for i in idxes]
labels_train = [labels_train[i] for i in idxes]
# validação
idxes = list(range(len(samples_validation)))
random.shuffle(idxes)
samples_validation = [samples_validation[i] for i in idxes]
labels_validation = [labels_validation[i] for i in idxes]
# teste
idxes = list(range(len(samples_test)))
random.shuffle(idxes)
samples_test = [samples_test[i] for i in idxes]
labels_test = [labels_test[i] for i in idxes]

In [12]:
# Salvando os datasets de treino, teste e validação
dir_datasets = '../datasets'
if not isdir(dir_datasets):
    mkdir(dir_datasets)
pickle.dump({'samples': samples_train, 'labels': labels_train}, open(join(dir_datasets,'train_dataset.pickle'), 'wb'))
pickle.dump({'samples': samples_validation, 'labels': labels_validation}, open(join(dir_datasets,'validation_dataset.pickle'), 'wb'))
pickle.dump({'samples': samples_test, 'labels': labels_test}, open(join(dir_datasets,'test_dataset.pickle'), 'wb'))

print(f'Arquivos salvos em {dir_datasets}')

Arquivos salvos em ../datasets
